<a href="https://colab.research.google.com/github/CMedrado/Topicosemcomputa-o/blob/main/Netbeans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classificação de Texto no contexto da Predição de Severidade de Bug Reports

Exemplo de classificação de texto com pipeline de NLP e classificador no final

In [3]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.simplefilter("ignore")

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix


In [4]:
# Tokenização
def tokenize(text):
    # Tokenização
    tokens = nltk.word_tokenize(text)
    
    # Stemização
    stems  = []
    for item in tokens:
        stems.append(SnowballStemmer("english").stem(item))
    return stems

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# Carregando os dados
df = pd.read_csv('/content/drive/My Drive/TP/PF/Dataset (1)/Netbeans_total.csv') # Salvamos com esse nome antes

# Dataset
df.head(11)

,Bug_report_ID,Product_Name,summary,description,component_name,severity_level,priority,assignee_developers,topic_id,status
0,236107,java,hint jdk profil,inspect transform dialog categori name jdk cat...,Hints,normal,P1,sdedic,geertjan sdedic sdedic,FIXED
1,10499,java,perform i n horribl slow popup local menu,i n tool action group deep tre scan return sta...,I18N,blocker,P1,issues,pnejedly pzavadsky vnemec jchalupa quality,FIXED
2,250700,java,except occur compil,creat attach actual error netbean ide develop ...,Compiler,normal,P1,dbalek,ShakirAli dbalek ShakirAli dbalek ShakirAli Sh...,FIXED
3,118348,java,cat correct code mark error,build jdk version line pobjectkey null mark er...,Compiler,blocker,P1,jlahoda,gugrim jbecicka gugrim medotin jlahoda jlahoda...,FIXED
4,19137,java,assign string compon resourc bundl corrupt res...,resourc bundl english german string form edito...,I18N,blocker,P1,issues,rost tpavek pkuzel pkuzel pzavadsky pkuzel pfe...,FIXED
5,226568,java,nullpointerexcept com sun tools javac code typ...,build netbean ide dev build vm java hotspot tm...,Compiler,normal,P1,dbalek,Chiana Chiana ecerichter sreimers ecerichter e...,FIXED
6,235702,java,implement perform bug hint,hint implement stringbuff replac string box bo...,Hints,normal,P1,sdedic,sdedic sdedic,FIXED
7,43936,java,except print consol creat ccdb jdk,current trunk reproduc test freeform project t...,Classfile,blocker,P1,tball,jglick jglick jglick jglick jglick tball jglic...,FIXED
8,53722,java,error compil test,compil error occur freeform test execut test u...,Freeform,blocker,P1,dkonecny,pzajac pzajac tzezula dkonecny dkonecny pzajac,FIXED
9,75619,java,editor speed packag explor invalid file,featur request editor speed mark problemat cod...,Editor,blocker,P1,issues,dyegoleal dyegoleal jbecicka dyegoleal,FIXED


In [7]:
df['severity_level'].value_counts()

blocker                                                                                            13747
normal                                                                                              7617
major                                                                                                 70
enhancement                                                                                           44
minor                                                                                                 35
critical                                                                                              13
trivial                                                                                               12
frankberger tzezula                                                                                    2
sdvic msauer mkubec jlahoda                                                                            2
saiandrag tzezula quality                              

In [8]:
class_names = df['severity_level'].unique()
class_names

array(['normal', 'blocker', 'major', nan,
       'rvbabilonia lcincura jbecicka tball tball ',
       'ozguy ozguy ozguy anebuzelsky jlahoda jlahoda ozguy pribyl jbecicka msauer sj-nb pflaska ',
       'pventura pventura dbalek ', 'jbatchelor jbatchelor jbecicka ',
       'enhancement', 'minor',
       'nixsa mkubec jbecicka phrebejk phrebejk phrebejk ',
       'ayeshasilvia mkubec ayeshasilvia tzezula tzezula jiriprox ',
       'ccpurvis tzezula ccpurvis ', 'kduz0 juhrik mmatula dsimonek ',
       'trivial', 'gustavomf gustavomf mkubec tzezula tzezula jiriprox ',
       'sdvic msauer mkubec jlahoda ',
       'bceglia bceglia lhasik lhasik asotona tzezula asotona nbabb asotona asotona lhasik lhasik ',
       'frankberger tzezula ', 'saiandrag tzezula quality ',
       'dhollon dhollon dhollon quality ',
       'rremedio rremedio jiriprox jiriprox jiriprox jiriprox jbecicka quality ',
       'critical',
       'vhi jglick bleonard bleonard jbecicka jbecicka quality ',
       'umafaid tz

In [9]:
status_names = df['status'].unique() 
status_names

array(['FIXED', nan], dtype=object)

In [10]:
#Convertendo 'summary' e 'description' para string
df = df.astype({'summary':'str', 'description':'str'})

In [11]:
#Filtrando severidades confiaveis
df = df.loc[(df['severity_level'] != 'enhancement') & (df['severity_level'] != 'normal')]

In [12]:
#Modificando severidade diferente de blocker (Classificação binaria)
df.loc[(df['severity_level'] != 'blocker'), 'severity_level'] = "no-blocker"

In [13]:
df['severity_level'].value_counts()

blocker       13747
no-blocker      917
Name: severity_level, dtype: int64

In [14]:
df.head(11)

,Bug_report_ID,Product_Name,summary,description,component_name,severity_level,priority,assignee_developers,topic_id,status
1,10499,java,perform i n horribl slow popup local menu,i n tool action group deep tre scan return sta...,I18N,blocker,P1,issues,pnejedly pzavadsky vnemec jchalupa quality,FIXED
3,118348,java,cat correct code mark error,build jdk version line pobjectkey null mark er...,Compiler,blocker,P1,jlahoda,gugrim jbecicka gugrim medotin jlahoda jlahoda...,FIXED
4,19137,java,assign string compon resourc bundl corrupt res...,resourc bundl english german string form edito...,I18N,blocker,P1,issues,rost tpavek pkuzel pkuzel pzavadsky pkuzel pfe...,FIXED
7,43936,java,except print consol creat ccdb jdk,current trunk reproduc test freeform project t...,Classfile,blocker,P1,tball,jglick jglick jglick jglick jglick tball jglic...,FIXED
8,53722,java,error compil test,compil error occur freeform test execut test u...,Freeform,blocker,P1,dkonecny,pzajac pzajac tzezula dkonecny dkonecny pzajac,FIXED
9,75619,java,editor speed packag explor invalid file,featur request editor speed mark problemat cod...,Editor,blocker,P1,issues,dyegoleal dyegoleal jbecicka dyegoleal,FIXED
10,60183,java,nsme jmiutils getexactclass string javahintspr...,org netbeans modules java hints org netbeans m...,Editor,blocker,P1,mmetelka,jglick jglick thurka thurka thurka jiriprox,FIXED
13,49026,java,permit user ignor list javac,recent re organ java fiel java exist sourc pro...,Classpath,blocker,P1,jglick,kjmcdonald jglick kjmcdonald jglick jglick jgl...,FIXED
16,41440,java,rethink ui i n wizard light buildsi,design add sourc dialog wizard strang consult ...,I18N,blocker,P1,mpetras,dkonecny mpetras dkonecny jglick pkuzel rypacek,FIXED
17,91888,java,highlight rewrit editor api,plan task,Editor,blocker,P1,jlahoda,phrebejk phrebejk jlahoda,FIXED


In [15]:
df.shape[0]

14664

In [16]:
# Stop Words

nltk.download('stopwords')

nltk.download('punkt')

stop_words = nltk.corpus.stopwords.words('english')

text_clf   = Pipeline([
                # Vectorize
                ('vect',  TfidfVectorizer(tokenizer=tokenize, 
                                          stop_words=stop_words, 
                                          ngram_range=(1,1))),
                # Classificador
                ('clf',   KNeighborsClassifier(n_jobs=-5)),
            ])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Divididos em 11 quadros não sobrepostos de tamanhos iguais.

In [17]:
df_set = []
k = 1333
for i in range(0,10):
  df_set.append(df.iloc[i*1333:k])
  k += 1333
  print("Quadro {}".format(i), "possui {} entradas.".format(df_set[i].shape[0]))

df_set.append(df.iloc[13330:14664])

print("Quadro 10 possui {} entradas.".format(df_set[10].shape[0]))

Quadro 0 possui 1333 entradas.
Quadro 1 possui 1333 entradas.
Quadro 2 possui 1333 entradas.
Quadro 3 possui 1333 entradas.
Quadro 4 possui 1333 entradas.
Quadro 5 possui 1333 entradas.
Quadro 6 possui 1333 entradas.
Quadro 7 possui 1333 entradas.
Quadro 8 possui 1333 entradas.
Quadro 9 possui 1333 entradas.
Quadro 10 possui 1334 entradas.


In [ ]:
X_test = []

for i in range(0, 10):
  new_df = df_set[0]
  for j in range(0, i):
    new_df = pd.merge(new_df, df_set[j+1], how = 'outer')
  X_test.append(new_df)

In [ ]:
for i in range(0, len(X_test)):
  print("Treinamento {}, possui {} entradas.". format(i+1, X_test[i].shape[0]))

In [ ]:
# Evaluate
for i in range(0, len(X_test)):
  text_clf = text_clf.fit(X_test[i].description, X_test[i].assignee_developers)
  pred = text_clf.predict(df_set[i+1].description)
  accuracy_scoreMLP_TFIDF = f1_score(df_set[i+1].assignee_developers, pred, average='micro')
  accuracy_scoreMLP_TFIDF

In [ ]:

predictions = text_clf.predict(df_set[10].description)

conf_matrix = confusion_matrix(df_set[10].assignee_developers, predictions)

In [ ]:
plt.figure(figsize=(10,10))
plt.xticks(np.arange(len(text_clf.classes_)),text_clf.classes_)
plt.yticks(np.arange(len(text_clf.classes_)),text_clf.classes_)
plt.imshow(conf_matrix,cmap=plt.cm.Blues)
plt.colorbar()